In [43]:
! pip install datasets classy-fire scikit-learn openai


[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
from datasets import load_dataset
from classy_fire import LLMClassifier
from dotenv import load_dotenv
from openai import InvalidRequestError
import random

# requires a .env file with OPENAI_API_BASE, OPENAI_API_VERSION, OPENAI_API_TYPE and OPENAI_API_KEY variables in the same directory as this notebook
load_dotenv()

# Loading an arbitrary text classification dataset from huggingface
dataset = load_dataset("tweet_eval", "emotion")

ds = dataset['test']
classifier = LLMClassifier(ds.features['label'].names,
                           task_description="The input is a tweet and the options are emotions that would best match the tweet.",
                          )
print(ds.features['label'].names)

['anger', 'joy', 'optimism', 'sadness']


In [70]:
classifier('That is the funniest thing I have ever heard!')

('joy', 1)

In [72]:
classifier('I miss you!')

('sadness', 3)

In [73]:
y_true, y_pred = [], []
random.seed(42)
for smpl in random.choices(ds, k=100):
    txt, y = smpl['text'], smpl['label']
    try:
        yhat = classifier(txt)[1]
    except InvalidRequestError:
        print(f'The following input failed: {txt}')
        continue
    y_true.append(y)
    y_pred.append(yhat)

The following input failed: My mother-in-law, in a fit of #rage, referred to someone as a 'yo-yo'.\n#yoyo #motherinlaw #amirite #fitofrage


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


The following input failed: 2Chainz my nigga 😊
The following input failed: @user @user Did you see it?  It's literally all porn.  Does she really work for huff post?
The following input failed: @user @user She is Right shld burn them alive 😡 it's even more better if they burn you with ur brothers 😠


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


In [74]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred, target_names=ds.features['label'].names, labels=list(set(y_pred))))

              precision    recall  f1-score   support

       anger       0.97      0.81      0.88        42
         joy       0.74      0.92      0.82        25
    optimism       1.00      0.25      0.40         8
     sadness       0.75      1.00      0.86        21

    accuracy                           0.83        96
   macro avg       0.87      0.74      0.74        96
weighted avg       0.87      0.83      0.82        96



At the time of writing, according to https://huggingface.co/spaces/autoevaluate/leaderboards?dataset=tweet_eval&only_verified=0&task=-any-&config=emotion&split=test&metric=f1
This beats the SOTA in leaderboard (on the sampled test data)